In [1]:
import tarfile

In [2]:
import json
import ast

In [3]:
import pandas as pd

In [4]:
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split
from scipy.sparse import coo_matrix as sp

C:\Users\Richa\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [5]:
my_tar = tarfile.open('C:/Users/Richa/Downloads/epinions_data.tar.gz')
my_tar.extractall('Gdata')
my_tar.close()

### Correcting JSON file

In [6]:
fr=open("C:/Users/Richa/Gdata/epinions_data/epinions.json")
fw=open("C:/Users/Richa/Gdata/epinions_data/epinions1.json", "w")

for line in fr:
    json_dat = json.dumps(ast.literal_eval(line))
    dict_dat = json.loads(json_dat)
    json.dump(dict_dat, fw)
    fw.write("\n")

fw.close()
fr.close()

In [7]:
with open('C:/Users/Richa/Gdata/epinions_data/epinions1.json') as json_file:
    data = json_file.readlines()
    data = list(map(json.loads, data))
epi=pd.DataFrame(data)
epi

,user,stars,time,paid,item,review
0,fgb59h,4.0,1027296000,260,Minolta_QMS_PagePro_1250E_Printers,i got this printer from minolta as a warranty ...
1,bucho_ky,2.0,1201305600,1800,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...,although i love my vaio especially the gorgeou...
2,redp944,4.0,1118016000,1000,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...,this is my first laptop and i bought it about ...
3,spongebag7,4.0,1149292800,20,pr-Durabrand_CD-85_Personal_CD_Player,just recently my sony dej001 walkman died on m...
4,pyros7,5.0,1012262400,1199,pr-Sony_KV_36FS12__Standard_Televisions,well when i graduated from college it didnt ta...
...,...,...,...,...,...,...
188473,chris_baehr,2.0,1012608000,100000,auto_Make-2002_Ford_Thunderbird,ford has once again tried to bring back the th...
188474,BrianCam,4.0,1016409600,44000,auto_Make-2002_Ford_Thunderbird,ford thunderbird the name alone says so much a...
188475,lanh,1.0,1051747200,41000,auto_Make-2002_Ford_Thunderbird,price is very high per features offered when y...
188476,spdster2002,5.0,1029974400,42155,auto_Make-2002_Ford_Thunderbird,after a 5year break ford finally restores its ...


In [8]:
epi.to_csv('C:/Users/Richa/Gdata/epinions_data/epinions2.csv')

In [9]:
general=pd.read_csv('C:/Users/Richa/Gdata/epinions_data/epinions2.csv')

In [10]:
general.head()

,Unnamed: 0,user,stars,time,paid,item,review
0,0,fgb59h,4.0,1027296000,260.0,Minolta_QMS_PagePro_1250E_Printers,i got this printer from minolta as a warranty ...
1,1,bucho_ky,2.0,1201305600,1800.0,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...,although i love my vaio especially the gorgeou...
2,2,redp944,4.0,1118016000,1000.0,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...,this is my first laptop and i bought it about ...
3,3,spongebag7,4.0,1149292800,20.0,pr-Durabrand_CD-85_Personal_CD_Player,just recently my sony dej001 walkman died on m...
4,4,pyros7,5.0,1012262400,1199.0,pr-Sony_KV_36FS12__Standard_Televisions,well when i graduated from college it didnt ta...


In [11]:
general=general.drop(['Unnamed: 0','time','paid','review'],axis=1)
general.head()

,user,stars,item
0,fgb59h,4.0,Minolta_QMS_PagePro_1250E_Printers
1,bucho_ky,2.0,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...
2,redp944,4.0,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...
3,spongebag7,4.0,pr-Durabrand_CD-85_Personal_CD_Player
4,pyros7,5.0,pr-Sony_KV_36FS12__Standard_Televisions


### Correcting Group dataset

In [12]:
gen1=pd.read_csv('C:/Users/Richa/Gdata/epinions_data/network_trust.txt',header=None)

In [13]:
gen2=pd.read_csv('C:/Users/Richa/Gdata/epinions_data/network_trustedby.txt',header=None)

In [14]:
gen1.to_csv('C:/Users/Richa/Gdata/epinions_data/gengroup1.csv')

In [15]:
gen2.to_csv('C:/Users/Richa/Gdata/epinions_data/gengroup2.csv')

In [16]:
data1=pd.read_csv('C:/Users/Richa/Gdata/epinions_data/gengroup1.csv')

In [17]:
data2=pd.read_csv('C:/Users/Richa/Gdata/epinions_data/gengroup2.csv')

In [18]:
data1.columns

Index(['Unnamed: 0', '0'], dtype='object')

In [19]:
data2.columns

Index(['Unnamed: 0', '0'], dtype='object')

In [20]:
data1.columns=['SNo','Groupx']
data1.columns

Index(['SNo', 'Groupx'], dtype='object')

In [21]:
data2.columns=['SNo','Groupx']
data2.columns

Index(['SNo', 'Groupx'], dtype='object')

In [22]:
new1 = data1["Groupx"].str.split(" ", n = -1, expand = True)

In [23]:
new2 = data2["Groupx"].str.split(" ", n = -1, expand = True)

In [24]:
data1["user"]= new1[0]
data1["trust"]= new1[1]
data1["Group"] = new1[2]
data1.drop(columns =["Groupx","SNo"], inplace = True)
data1.head()

,user,trust,Group
0,lavenderfruit,trust,kristinafh
1,shippo225,trust,sallyg8
2,aryeh7000,trust,liteair
3,chiquita,trust,magenta321
4,jeffsullivan,trust,rpllingrock


In [25]:
data1.count()

user     691895
trust    691895
Group    691895
dtype: int64

In [26]:
data2["Group"]= new2[0]
data2["trustedby"]= new2[1]
data2["user"] = new2[2]
data2.drop(columns =["Groupx","SNo"], inplace = True)
data2.head()

,Group,trustedby,user
0,lavenderfruit,trustedby,kristinafh
1,Rosalyce,trustedby,cindee
2,Sharkie,trustedby,theeye
3,chiquita,trustedby,magenta321
4,ValerieG,trustedby,jabbrjwz2


In [27]:
data2.count()

Group        691895
trustedby    691895
user         691895
dtype: int64

In [28]:
data = pd.concat([data1,data2])
data.head()

,user,trust,Group,trustedby
0,lavenderfruit,trust,kristinafh,NaN
1,shippo225,trust,sallyg8,NaN
2,aryeh7000,trust,liteair,NaN
3,chiquita,trust,magenta321,NaN
4,jeffsullivan,trust,rpllingrock,NaN


In [29]:
data.count()

user         1383790
trust         691895
Group        1383790
trustedby     691895
dtype: int64

In [30]:
data1.to_csv('C:/Users/Richa/Gdata/epinions_data/gengroup1.csv')
data2.to_csv('C:/Users/Richa/Gdata/epinions_data/gengroup2.csv')

In [31]:
data1=pd.read_csv('C:/Users/Richa/Gdata/epinions_data/gengroup1.csv')
data2=pd.read_csv('C:/Users/Richa/Gdata/epinions_data/gengroup2.csv')

In [32]:
data=data.drop(['trust','trustedby'],axis=1)

In [33]:
#removing duplicate values
x = data.duplicated().sum()
print("No of Duplicate Values", x)
data.drop_duplicates(inplace=True)
y = data.duplicated().sum()
print("No of Duplicate Values after cleaning", y)

No of Duplicate Values 224274
No of Duplicate Values after cleaning 0


In [34]:
data.count()

user     1159516
Group    1159516
dtype: int64

In [35]:
data.to_csv('C:/Users/Richa/Gdata/epinions_data/gengroup.csv')

In [36]:
data=pd.read_csv('C:/Users/Richa/Gdata/epinions_data/gengroup.csv')

In [37]:
data.head()

,Unnamed: 0,user,Group
0,0,lavenderfruit,kristinafh
1,1,shippo225,sallyg8
2,2,aryeh7000,liteair
3,3,chiquita,magenta321
4,4,jeffsullivan,rpllingrock


In [38]:
data.columns

Index(['Unnamed: 0', 'user', 'Group'], dtype='object')

In [39]:
data=data.drop(['Unnamed: 0'],axis=1)

### Combining Tables

In [40]:
combinedTable1=pd.merge(general,data,on='user',how='left')

In [41]:
combinedTable1.head()

,user,stars,item,Group
0,fgb59h,4.0,Minolta_QMS_PagePro_1250E_Printers,NaN
1,bucho_ky,2.0,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...,NaN
2,redp944,4.0,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...,NaN
3,spongebag7,4.0,pr-Durabrand_CD-85_Personal_CD_Player,cyndia
4,spongebag7,4.0,pr-Durabrand_CD-85_Personal_CD_Player,amyk49


In [42]:
#removing duplicate values
x = combinedTable1.duplicated().sum()
print("No of Duplicate Values", x)
combinedTable1.drop_duplicates(inplace=True)
y = combinedTable1.duplicated().sum()
print("No of Duplicate Values after cleaning", y)

No of Duplicate Values 2097
No of Duplicate Values after cleaning 0


In [43]:
combinedTable1.count()

user     2755804
stars    2756140
item     2756140
Group    2622806
dtype: int64

In [44]:
combinedTable1=combinedTable1.dropna()

In [45]:
combinedTable1.count()

user     2622470
stars    2622470
item     2622470
Group    2622470
dtype: int64

In [46]:
combinedTable1.head()

,user,stars,item,Group
3,spongebag7,4.0,pr-Durabrand_CD-85_Personal_CD_Player,cyndia
4,spongebag7,4.0,pr-Durabrand_CD-85_Personal_CD_Player,amyk49
5,spongebag7,4.0,pr-Durabrand_CD-85_Personal_CD_Player,rated-r
6,spongebag7,4.0,pr-Durabrand_CD-85_Personal_CD_Player,trailhound
7,spongebag7,4.0,pr-Durabrand_CD-85_Personal_CD_Player,megugrrrl


In [47]:
combinedTable1.to_csv('C:/Users/Richa/Gdata/epinions_data/combinedTable.csv')

In [48]:
combinedTable1=pd.read_csv('C:/Users/Richa/Gdata/epinions_data/combinedTable.csv')

In [49]:
print(combinedTable1.user.nunique(),combinedTable1.item.nunique(),combinedTable1.Group.nunique())

17607 25911 49366


In [50]:
data1=combinedTable1.copy(deep=True)

### Prepare the data

In [51]:
dataset=Dataset()

In [52]:
dataset.fit(data1.user.unique(), data1.item.unique())

In [53]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 17607, num_items 25911.


In [54]:
interactions, weights = dataset.build_interactions([tuple(i) for i in data1[['user','item','stars']].values])